<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(143)
df = pd.DataFrame({
    'x1': [0, 0, 1, 0, 1, 1, 0],
    'x2': [0, 1, 0, 1, 0, 1, 0],
    'x3': [1, 1, 1, 0, 0, 1, 0],
    'y': [0, 1, 1, 1, 1, 0, 0]
})
df.head()

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1


In [2]:
X = np.array(df[['x1', 'x2', 'x3']].values)
y = np.array(df['y'].apply(lambda _: [_]).tolist())
X, y

(array([[0, 0, 1],
        [0, 1, 1],
        [1, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 1, 1],
        [0, 0, 0]]), array([[0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0]]))

In [3]:
class NeuralNetwork:
    def __init__(self, i, h, o):
        self.inputs = i
        self.hiddens = h
        self.outputs = o
        self.weights1 = np.random.rand(self.inputs, self.hiddens)
        self.weights2 = np.random.rand(self.hiddens, self.outputs)
    
    def __sigmoid(self, X):
        return 1 / (1 + np.exp(-X))
    def __sigderiv(self, X):
        sx = self.__sigmoid(X)
        return sx * (1 - sx)
    
    def feed_forward(self, X):
        self.weight_sum = np.dot(X, self.weights1)
        self.activated1 = self.__sigmoid(self.weight_sum)
        self.output = np.dot(self.activated1, self.weights2)
        self.activated2 = self.__sigmoid(self.output)
        return self.activated2
    
    def back_prop(self, X, y):
        o = self.activated2
        self.o_error = y - o
        self.o_delta = self.o_error * self.__sigderiv(o)
        self.z2_error = self.o_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.__sigderiv(self.activated1)
        self.loss = np.mean(np.square(self.o_error))
        self.weights1 += X.T.dot(self.z2_delta)
        self.weights2 += self.activated1.T.dot(self.o_delta)
    
    def train(self, X, y):
        self.feed_forward(X)
        self.back_prop(X, y)
    
    def train_lots(self, X, y):
        while True:
            self.train(X, y)
            if self.loss < 0.001:
                break

In [4]:
nn = NeuralNetwork(3, 4, 1)
nn.train_lots(X, y)

In [5]:
nn.feed_forward(X)

array([[0.02787232],
       [0.95552895],
       [0.95467742],
       [0.99270408],
       [0.99402979],
       [0.04346544],
       [0.01421585]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [6]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [7]:
# input image dimensions
img_rows, img_cols = 28, 28

In [8]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [10]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [11]:
y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [12]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [ ]:
nn2 = NeuralNetwork(784, 2, 1)
nn2.train(x_train, y_train)

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?